#### attempting to replicate photonvision's best practices for my own pipeline

In [1]:
# Cell 1: Setup & Constants
import cv2
import numpy as np
import time
import math
import logging

# We will use robotpy_apriltag to get the official field layout
# This ensures our "Object Points" match the official game data exactly.
from robotpy_apriltag import AprilTagField, AprilTagFieldLayout

# Standard FRC Coordinate Systems
# WPILib: X=Forward, Y=Left, Z=Up
# OpenCV: Z=Forward, X=Right, Y=Down
# We will define the transform matrices explicitly to avoid confusion.

class PhotonConstants:
    # Standard AprilTag 36h11 size for FRC (16.51 cm)
    TAG_SIZE_METERS = 0.1651 
    
    # 2025 Reefscape Layout (fallback to Crescendo if not found)
    try:
        FIELD_LAYOUT = AprilTagFieldLayout.loadField(AprilTagField.k2025Reefscape)
        print("Loaded 2025 Reefscape Layout")
    except Exception:
        FIELD_LAYOUT = AprilTagFieldLayout.loadField(AprilTagField.k2024Crescendo)
        print("Loaded 2024 Crescendo Layout")

    # Corner offsets in Tag-Local space (Standard Counter-Clockwise from Bottom-Left in WPILib?)
    # WAIT: OpenCV expects corners Top-Left, Top-Right, Bottom-Right, Bottom-Left
    # We must match the order the detector returns.
    s = TAG_SIZE_METERS / 2.0
    
    # These are the 3D points of a tag centered at (0,0,0) facing +X in WPILib coords
    # But OpenCV solvePnP usually assumes the object is in the XY plane (Z=0).
    # PhotonVision defines corner locations in the *Field Coordinate System* directly for Multi-Tag.
    
    # For single tag solve, we define the tag in its own reference frame:
    # (Z forward, X right, Y down - standard OpenCV local frame)
    # This matches the standard solvePnP expectation.
    TAG_MODEL_OPENCV = np.array([
        [-s, -s, 0.0], # Top-Left
        [ s, -s, 0.0], # Top-Right
        [ s,  s, 0.0], # Bottom-Right
        [-s,  s, 0.0]  # Bottom-Left
    ], dtype=np.float64)

# Verification
print(f"Tag Model Shape: {PhotonConstants.TAG_MODEL_OPENCV.shape}")

Loaded 2024 Crescendo Layout
Tag Model Shape: (4, 3)


In [25]:
# Cell 1 (Revised): Imports & State
import cv2
import numpy as np
import time
from wpimath.geometry import Pose3d, Rotation3d, Translation3d, Transform3d, Quaternion

# Global State for History Tracking
class VisionState:
    last_robot_pose = None # Pose3d
    last_timestamp = 0.0
    
print("Imports fixed. State container ready.")

Imports fixed. State container ready.


In [20]:
# Cell 2: Coordinate Helpers
from wpimath.geometry import Transform3d, Translation3d, Rotation3d, Pose3d, Quaternion

class VisionMath:
    @staticmethod
    def get_cam_to_robot_pose(cam_pose_in_field: Pose3d, camera_to_robot: Transform3d) -> Pose3d:
        """
        Converts the camera's position in the field to the robot's position.
        cam_pose_in_field: Where the camera is (in WPILib field coords)
        camera_to_robot: The transform FROM camera TO robot (inverse of robot_to_cam)
        """
        return cam_pose_in_field.transformBy(camera_to_robot)

    @staticmethod
    def open_cv_to_wpilib(tvec, rvec):
        """
        Converts OpenCV solvePnP output (Camera->Tag) to WPILib (Camera->Tag).
        OpenCV: Z-forward, X-right, Y-down
        WPILib: X-forward, Y-left, Z-up
        """
        # Rotation Matrix (OpenCV)
        R_cv, _ = cv2.Rodrigues(rvec)
        
        # Transformation Matrix to switch axes
        # We want to map:
        # CV_Z (Forward) -> WPI_X (Forward)
        # CV_X (Right)   -> WPI_Y (Left) * -1 
        # CV_Y (Down)    -> WPI_Z (Up) * -1
        # This is a standard coordinate basis change.
        # However, PhotonVision often does this by transforming the *result* pose.
        
        # Alternative (PhotonVision style):
        # 1. Get the translation in Camera Frame
        x = tvec[2][0]
        y = -tvec[0][0]
        z = -tvec[1][0]
        
        # 2. Convert Rotation
        # This is the tricky part. We usually construct a Rotation3d from the matrix
        # then rotate it to match the WPILib frame.
        # For this notebook, we will write a rigorous test in Cell 5 to prove this transform.
        return x, y, z

    @staticmethod
    def sort_corners(corners):
        """
        Ensure corners are always [TL, TR, BR, BL] based on the image axes.
        This fixes the 'permutation' bugs seen in custom pipelines.
        """
        # Center of mass
        center = np.mean(corners, axis=0)
        
        # Sort by angle to ensure counter-clockwise (or clockwise) order
        # Then rotate the array so the point with (min X + min Y) is first (Top-Left)
        # For simplicity in this cell, we will trust the detector's winding 
        # but ensure the "first" corner is the Top-Left one visually.
        
        # Simple heuristic for standard upright tags:
        # Top-Left has the smallest (x + y) sum.
        sums = [pt[0] + pt[1] for pt in corners]
        tl_idx = np.argmin(sums)
        
        return np.roll(corners, -tl_idx, axis=0)

print("Math Helpers Defined.")

Math Helpers Defined.


In [19]:
# Cell 3: Single Tag Solver (Strict IPPE Compliance)
import cv2
import numpy as np

def solve_single_tag(corners, K, D, tag_size):
    """
    Solves for single tag pose using SOLVEPNP_IPPE_SQUARE.
    
    CRITICAL: IPPE_SQUARE requires specific corner order:
    0: [-s/2,  s/2] -> Bottom-Left (in OpenCV Y-down)
    1: [ s/2,  s/2] -> Bottom-Right
    2: [ s/2, -s/2] -> Top-Right
    3: [-s/2, -s/2] -> Top-Left
    
    Args:
        corners: (4, 2) numpy array. EXPECTED ORDER: [TL, TR, BR, BL]
    """
    s = tag_size
    # Defined strictly by the IPPE_SQUARE docstring
    object_points = np.array([
        [-s/2,  s/2, 0], # Point 0: Bottom-Left
        [ s/2,  s/2, 0], # Point 1: Bottom-Right
        [ s/2, -s/2, 0], # Point 2: Top-Right
        [-s/2, -s/2, 0]  # Point 3: Top-Left
    ], dtype=np.float64).reshape(4, 1, 3)

    # Reorder Image Points to match Object Points
    # Input assumed: [TL(0), TR(1), BR(2), BL(3)]
    # Target:        [BL(3), BR(2), TR(1), TL(0)]
    # Note: If input is [TL, TR, BR, BL], then:
    # BL is index 3
    # BR is index 2
    # TR is index 1
    # TL is index 0
    img_ordered = corners[[3, 2, 1, 0], :].astype(np.float64).reshape(4, 1, 2)

    K = K.astype(np.float64)
    D = D.astype(np.float64).reshape(-1, 1)

    try:
        # Strict unpacking based on user docstring: retval, rvecs, tvecs, errs
        ret, rvecs, tvecs, errs = cv2.solvePnPGeneric(
            object_points, img_ordered, K, D, flags=cv2.SOLVEPNP_IPPE_SQUARE
        )
    except Exception as e:
        print(f"Solver Error: {e}")
        return None

    if len(rvecs) == 0: return None

    # Calculate Ambiguity
    # IPPE returns 2 solutions. We want the one with lowest error.
    # Note: 'errs' from solvePnPGeneric corresponds to the reordered points.
    
    # We must trust our own reprojection check because 'errs' behavior varies by version
    actual_errors = []
    # Project back to verify (using the IPPE object point order)
    obj_pts_flat = object_points.reshape(4, 3)
    
    for i in range(len(rvecs)):
        proj, _ = cv2.projectPoints(obj_pts_flat, rvecs[i], tvecs[i], K, D)
        err = cv2.norm(img_ordered.reshape(4,2), proj.reshape(4,2), cv2.NORM_L2)
        actual_errors.append(err)

    best_idx = np.argmin(actual_errors)
    best_err = actual_errors[best_idx]
    
    # Ambiguity Score
    if len(actual_errors) > 1:
        alt_idx = 1 - best_idx
        alt_err = actual_errors[alt_idx]
        ambiguity = 1.0 if alt_err < 1e-6 else best_err / alt_err
    else:
        ambiguity = 0.0

    return {
        'rvec': rvecs[best_idx], 
        'tvec': tvecs[best_idx], 
        'error': best_err, 
        'ambiguity': ambiguity
    }

print("Cell 3: Single Tag Solver (Strict IPPE Spec) Defined.")

Cell 3: Single Tag Solver (Strict IPPE Spec) Defined.


In [26]:
# Cell 3 (Advanced): History-Aware Solver
def solve_single_tag_robust(corners, K, D, tag_size, prev_pose=None):
    """
    Solves IPPE and uses history (prev_pose) to pick the correct solution 
    if ambiguity is high.
    
    prev_pose: Pose3d (The Robot's field pose from the last frame)
    """
    s = tag_size
    # IPPE Order: BL, BR, TR, TL
    object_points = np.array([
        [-s/2,  s/2, 0], [ s/2,  s/2, 0], [ s/2, -s/2, 0], [-s/2, -s/2, 0]
    ], dtype=np.float64).reshape(4, 1, 3)

    # Detector usually returns TL, TR, BR, BL (Counter-Clockwise from TL)
    # We map Detector(0)->IPPE(3), Det(1)->IPPE(2), Det(2)->IPPE(1), Det(3)->IPPE(0)
    # Wait, simple reversal? 
    # Detector: TL, TR, BR, BL
    # IPPE:     BL, BR, TR, TL
    # Yes, it is exactly reversed order.
    img_ordered = corners[[3, 2, 1, 0], :].astype(np.float64).reshape(4, 1, 2)
    
    K = K.astype(np.float64); D = D.astype(np.float64).reshape(-1, 1)

    try:
        ret, rvecs, tvecs, _ = cv2.solvePnPGeneric(
            object_points, img_ordered, K, D, flags=cv2.SOLVEPNP_IPPE_SQUARE
        )
    except Exception: return None

    # ... (Unpacking logic from before) ...
    # (Simplified for brevity, assume valid rvecs found)
    
    solutions = []
    for i in range(len(rvecs)):
        # Calculate Error
        proj, _ = cv2.projectPoints(object_points, rvecs[i], tvecs[i], K, D)
        err = cv2.norm(img_ordered.reshape(4,2), proj.reshape(4,2), cv2.NORM_L2)
        
        # Calculate "Distance to History" if available
        dist_score = float('inf')
        if prev_pose is not None:
            # We need to convert this rvec/tvec to a Robot Field Pose to compare.
            # This requires the Field Layout and Camera Offset, which we don't have inside this function.
            # So PhotonVision typically returns BOTH solutions and lets the upper manager decide.
            pass
        
        solutions.append({'rvec': rvecs[i], 'tvec': tvecs[i], 'err': err})

    # Sort by Reprojection Error first
    solutions.sort(key=lambda x: x['err'])
    
    best = solutions[0]
    alt  = solutions[1] if len(solutions) > 1 else best
    
    # AMBIGUITY CHECK
    ratio = best['err'] / (alt['err'] + 1e-9)
    
    # If ratio is high (e.g. > 0.15), we are ambiguous.
    # IF ambiguous AND we have history:
    #   Pick the solution closest to history, even if its error is slightly worse.
    
    return solutions # Return ALL candidates so the manager can decide using history

print("History Logic: Concept defined.")

History Logic: Concept defined.


In [21]:
# Cell 4: Verification (Strict)
fx, fy, cx, cy = 500.0, 500.0, 320.0, 240.0
K_sim = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=np.float64)
D_sim = np.zeros(5)

# 1. Define Standard Object for generating "Perfect" corners
# We generate them as TL, TR, BR, BL (Standard visual order)
s = 0.1651 / 2.0
gen_obj = np.array([
    [-s, -s, 0], # TL
    [ s, -s, 0], # TR
    [ s,  s, 0], # BR
    [-s,  s, 0]  # BL
], dtype=np.float64)

# 2. Known Pose (2m away, 45 deg turn)
known_tvec = np.array([[0.0], [0.0], [2.0]], dtype=np.float64)
known_rvec = np.array([[0.0], [np.radians(45)], [0.0]], dtype=np.float64)

# 3. Project to get [TL, TR, BR, BL] pixels
perfect_corners, _ = cv2.projectPoints(gen_obj, known_rvec, known_tvec, K_sim, D_sim)
perfect_corners = perfect_corners.reshape(4, 2)

# 4. Solve
result = solve_single_tag(perfect_corners, K_sim, D_sim, 0.1651)

print(f"--- Test 1: 45 Degree Angle ---")
if result:
    print(f"Input Z: {known_tvec[2][0]:.4f} m | Solved Z: {result['tvec'][2][0]:.4f} m")
    print(f"Input Ry: {np.degrees(known_rvec[1][0]):.2f} deg | Solved Ry: {np.degrees(result['rvec'][1][0]):.2f} deg")
    print(f"Ambiguity: {result['ambiguity']:.4f}")
    
    # Assertions
    assert np.isclose(result['tvec'][2][0], 2.0, atol=0.05), f"Depth Wrong: {result['tvec'][2][0]}"
    assert result['ambiguity'] < 0.2, "Ambiguity too high for clear target"
    print("PASSED: 45 Deg Test")
else:
    print("FAILED: Solver returned None")

# 5. Face-On Test (High Ambiguity)
face_rvec = np.array([[0.0], [0.0], [0.0]], dtype=np.float64)
face_corners, _ = cv2.projectPoints(gen_obj, face_rvec, known_tvec, K_sim, D_sim)
face_corners = face_corners.reshape(4, 2)
# Add noise to induce ambiguity
noise = np.random.normal(0, 0.5, face_corners.shape) 
result_bad = solve_single_tag(face_corners + noise, K_sim, D_sim, 0.1651)

print(f"\n--- Test 2: Face-On (Noisy) ---")
if result_bad:
    print(f"Ambiguity: {result_bad['ambiguity']:.4f}")
    if result_bad['ambiguity'] > 0.1:
        print("PASSED: Ambiguity detected properly.")
    else:
        print("WARNING: Ambiguity low, but noise was random.")

--- Test 1: 45 Degree Angle ---
Input Z: 2.0000 m | Solved Z: 2.0000 m
Input Ry: 45.00 deg | Solved Ry: 45.00 deg
Ambiguity: 0.0000
PASSED: 45 Deg Test

--- Test 2: Face-On (Noisy) ---
Ambiguity: 0.1638
PASSED: Ambiguity detected properly.


In [22]:
# Cell 5: Multi-Tag Solver (PhotonVision Strategy)
import cv2
import numpy as np
from wpimath.geometry import Pose3d, Rotation3d, Translation3d, Transform3d

def solve_multi_tag(detections, field_layout, K, D):
    """
    Performs a single rigid-body solve using all visible tags.
    
    Args:
        detections: List of dicts {'id': int, 'corners': (4,2) np array [TL, TR, BR, BL]}
        field_layout: RobotPy AprilTagFieldLayout
        K, D: Camera intrinsics
        
    Returns:
        dict: {
            'rvec': np.array (Camera->Field), 
            'tvec': np.array (Camera->Field), 
            'rmse': float,
            'robot_pose': Pose3d (Field->Robot)
        }
        or None if solve fails.
    """
    obj_points = []
    img_points = []
    
    # Define Corner Offsets in WPILib Tag Frame (X=Forward, Y=Left, Z=Up)
    # Order matches standard detection: TL, TR, BR, BL
    s = 0.1651 / 2.0
    local_corners = [
        Translation3d(0.0,  s,  s), # TL (Up-Left)
        Translation3d(0.0, -s,  s), # TR (Up-Right)
        Translation3d(0.0, -s, -s), # BR (Down-Right)
        Translation3d(0.0,  s, -s), # BL (Down-Left)
    ]

    for det in detections:
        tag_id = int(det['id'])
        corners = det['corners'] # Expecting (4, 2)
        
        # Look up tag pose in field
        tag_pose = field_layout.getTagPose(tag_id)
        if tag_pose is None: continue

        # Transform local tag corners to Global Field Coordinates
        for i, pt_local in enumerate(local_corners):
            # corner_field = TagPose * corner_local
            global_pt = tag_pose.transformBy(Transform3d(pt_local, Rotation3d()))
            
            obj_points.append([global_pt.X(), global_pt.Y(), global_pt.Z()])
            img_points.append(corners[i])

    # Require at least 2 tags (8 points) for the "Multi-Tag Advantage"
    # (Technically works with 1, but we prefer single-tag IPPE for that)
    if len(obj_points) < 8: 
        return None
        
    obj_points = np.array(obj_points, dtype=np.float64)
    img_points = np.array(img_points, dtype=np.float64)
    K = K.astype(np.float64)
    D = D.astype(np.float64).reshape(-1, 1)

    # Solve FIELD -> CAMERA transform
    # SQPNP is more robust than ITERATIVE for multiple planar targets
    try:
        success, rvec, tvec = cv2.solvePnP(
            obj_points, img_points, K, D, flags=cv2.SOLVEPNP_SQPNP
        )
    except cv2.error:
        return None
    
    if not success: return None

    # Calculate Reprojection Error (RMSE)
    proj, _ = cv2.projectPoints(obj_points, rvec, tvec, K, D)
    err = cv2.norm(img_points.reshape(-1,2), proj.reshape(-1,2), cv2.NORM_L2)
    rmse = err / np.sqrt(len(obj_points))

    # Convert Result to Robot Pose
    # 1. Get Camera Pose in Field (C_field)
    #    SolvePnP gives T_field_to_camera (X_cam = R * X_field + t)
    #    We need X_field = R^T * (X_cam - t) -> Camera position is -R^T * t
    R_cv, _ = cv2.Rodrigues(rvec)
    t_field_to_cam = tvec.reshape(3, 1)
    
    # Camera Position in Field (XYZ)
    C_field = -R_cv.T @ t_field_to_cam
    
    # Camera Rotation in Field
    # R_cv maps Field(XYZ) -> Camera(Right, Down, Fwd)
    # We want Field(XYZ) -> Camera(Fwd, Left, Up) (WPILib)
    # R_wpi = P * R_cv  where P permutates axes
    # Actually, simpler to construct the Basis Vectors:
    # Cam Forward (X_wpi) is Z_cv (Column 2 of R_cv^T)
    # Cam Left    (Y_wpi) is -X_cv (Column 0 of R_cv^T * -1)
    # Cam Up      (Z_wpi) is -Y_cv (Column 1 of R_cv^T * -1)
    
    R_f2c_T = R_cv.T
    c_fwd =  R_f2c_T[:, 2] # Camera Z axis expressed in Field Coords
    c_up  = -R_f2c_T[:, 1] # Camera -Y axis expressed in Field Coords

    # Construct LookAt-style Rotation using WPILib geometry? 
    # Or just return the raw translation for now and handle rotation in the helper.
    # Let's do a direct construct for completeness.
    
    # For now, let's return the raw opencv components and the computed field translation
    # to keep the "Solver" pure math.
    
    return {
        'rvec': rvec,
        'tvec': tvec,
        'rmse': rmse,
        'field_translation': C_field.flatten()
    }
    
print("Cell 5: Multi-Tag Solver Defined.")

Cell 5: Multi-Tag Solver Defined.


In [24]:
# Cell 6: Rigorous Multi-Tag Verification
import numpy as np
import cv2
from wpimath.geometry import Pose3d, Rotation3d, Translation3d, Transform3d

# --- 1. Setup Ground Truth ---
# Camera is at Origin (0,0,0) in Field, facing +X direction.
# We must define the Transform from Field -> Camera for the projection.
# Field +X = Camera +Z (Forward)
# Field +Y = Camera -X (Left)
# Field +Z = Camera -Y (Up)
# This rotation matrix aligns the axes.
R_field_to_cam_gt = np.array([
    [ 0, -1,  0], # Field X -> Cam ? No.
    [ 0,  0, -1],
    [ 1,  0,  0]
], dtype=np.float64)
# Let's verify: 
# Field(1,0,0) -> R * [1,0,0] = [0,0,1] (Cam Z). Correct.
# Field(0,1,0) -> R * [0,1,0] = [-1,0,0] (Cam -X). Correct.
# Field(0,0,1) -> R * [0,0,1] = [0,-1,0] (Cam -Y). Correct.

t_field_to_cam_gt = np.zeros((3, 1), dtype=np.float64) # Camera is at 0,0,0

# Camera Intrinsics
K_sim = np.array([[500, 0, 320], [0, 500, 240], [0, 0, 1]], dtype=np.float64)
D_sim = np.zeros((5,1), dtype=np.float64)

# --- 2. Setup Mock Layout (Two Tags) ---
# Tag 1: 2m in front (X=2), facing Camera (-X).
t1_pose = Pose3d(Translation3d(2.0, 0.0, 0.0), Rotation3d(0, 0, np.pi))
# Tag 2: 2m in front, 1m Right (X=2, Y=-1), facing Camera.
t2_pose = Pose3d(Translation3d(2.0, -1.0, 0.0), Rotation3d(0, 0, np.pi))

class MockLayoutRigorous:
    def getTagPose(self, id):
        if id == 1: return t1_pose
        if id == 2: return t2_pose
        return None
layout_gt = MockLayoutRigorous()

# --- 3. Generate Perfect Pixel Inputs ---
# We use the SAME logic as the solver to get Field Points, 
# then project them using our known Ground Truth Camera.

detections_gt = []
s = 0.1651 / 2.0
local_corners = [
    Translation3d(0.0,  s,  s), # TL
    Translation3d(0.0, -s,  s), # TR
    Translation3d(0.0, -s, -s), # BR
    Translation3d(0.0,  s, -s), # BL
]

for tid in [1, 2]:
    pose = layout_gt.getTagPose(tid)
    obj_pts_field = []
    
    for pt_local in local_corners:
        # Convert Tag-Local -> Field
        global_pt = pose.transformBy(Transform3d(pt_local, Rotation3d()))
        obj_pts_field.append([global_pt.X(), global_pt.Y(), global_pt.Z()])
    
    obj_pts_field = np.array(obj_pts_field, dtype=np.float64).reshape(-1, 3)
    
    # Project Field -> Camera (using Ground Truth)
    # Note: projectPoints expects T_cam_to_obj? No, T_obj_to_cam.
    # We have T_field_to_cam.
    rvec_gt, _ = cv2.Rodrigues(R_field_to_cam_gt)
    img_pts, _ = cv2.projectPoints(obj_pts_field, rvec_gt, t_field_to_cam_gt, K_sim, D_sim)
    
    detections_gt.append({'id': tid, 'corners': img_pts.reshape(4, 2)})

# --- 4. Run The Solver ---
result = solve_multi_tag(detections_gt, layout_gt, K_sim, D_sim)

print("--- Rigorous Multi-Tag Verification ---")
if result:
    pos = result['field_translation']
    print(f"Solved Camera Pos: [{pos[0]:.4f}, {pos[1]:.4f}, {pos[2]:.4f}]")
    print(f"Expected Pos:      [0.0000, 0.0000, 0.0000]")
    
    err = np.linalg.norm(pos)
    print(f"Total Error:       {err:.6f} m")
    
    assert err < 0.01, f"Verification Failed. Error: {err}"
    print("PASSED: Solver matches Ground Truth.")
else:
    print("FAILED: Solver returned None")

--- Rigorous Multi-Tag Verification ---
Solved Camera Pos: [-0.0000, -0.0000, -0.0000]
Expected Pos:      [0.0000, 0.0000, 0.0000]
Total Error:       0.000000 m
PASSED: Solver matches Ground Truth.
